* PPGI
* Computação Científica com Julia
* Prof. José E. Storópoli
* Aluno: André Santos

In [1]:
# Packages requires to load the dataset
using Pkg
Pkg.add("DataFrames")
using DataFrames

    Updating registry at `C:\Users\andre\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\andre\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\andre\.julia\environments\v1.7\Manifest.toml`


In [2]:
date = [1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,
    2012,2013,2014,2015,2016,2017,2018,2019];
    
pop = [2,1,4,2,1,4,2,2,1,6,7,7,8,8,8,8,11,11,13,21,21,36,24,41,46,46,59,53];
df = DataFrame( varx = date, vary = pop);

## Get the sum of the squares of the residuals (total, before and after break point) 

In [3]:
# Define time series for Chow test:
#point break = 2010
start_df1, end_df1 = 1992, 2010                                      # range for series 1 (obs:"end_df1 = point break")
start_df2, end_df2 = end_df1+1, 2019                                 # range for series 2
df1 = filter(r -> r.varx >= start_df1 && r.varx <= end_df1, df)      # time series before point break
df2 = filter(r -> r.varx >= start_df2 && r.varx <= end_df2, df)      # time series after point break

,varx,vary
,Int64,Int64
1,2011,21
2,2012,21
3,2013,36
4,2014,24
5,2015,41
6,2016,46
7,2017,46
8,2018,59
9,2019,53


In [4]:
# Variables for Chow test:
x , y = copy(df[:,:varx]), copy(df[:,:vary])         # variables from original dataset
x1 , y1 = copy(df1[:,:varx]), copy(df1[:,:vary])     # variables from series before point break
x2 , y2 = copy(df2[:,:varx]), copy(df2[:,:vary])     # variables from series after point break

([2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019], [21, 21, 36, 24, 41, 46, 46, 59, 53])

In [5]:
# Alternative formulas ***************************************************************************************************
# Original dataset ----------------------------------------------------------------------------------------------- 
k = 2                                        # number of samples (series before and after point break)
n = length(x)                                # total scores from original dataset
sum_x, sum_y = sum(x), sum(y)                # sum of independent and dependent variables from original series
mean_x, mean_y = sum(x)/n, sum(y)/n          # average of independent and dependent variables from original series
SSxy = sum((x.-mean_x).*(y.-mean_y))         # covariance of X and Y
SSxx = sum((x.-mean_x).^2)                   # variance in X
b = SSxy/SSxx                                # regression coefficient
a = mean_y - b*mean_x                        # intercept
yi = a .+ b.*x                               # estimated (or predicted) y value for observation i
RSSp = sum((y .- yi).^2)                     # Resiual sum of square (errors)
# Series 1 ------------------------------------------------------------------------------------------------------ 
n1 = length(x1)                              # total scores from series 1
gl1 = n1 - k                                 # numerator degrees freedom
sum_x1, sum_y1 = sum(x1), sum(y1)            # sum of independent and dependent variables from series 1
mean_x1, mean_y1 = sum(x1)/n1, sum(y1)/n1    # average of independent and dependent variables from series 1
SSxy1 = sum((x1.-mean_x1).*(y1.-mean_y1))    # covariance of X1 and Y1
SSxx1 = sum((x1.-mean_x1).^2)                # variance in X1
b1 = SSxy1/SSxx1                             # regression coefficient (series 1)
a1 = mean_y1 - b1*mean_x1                    # intercept (series 1)
yi1 = a1 .+ b1.*x1                           # estimated y value for observation i (series 1)
RSS1 = sum((y1 .- yi1).^2)                   # Residual sum of square before break
# Series 2 ------------------------------------------------------------------------------------------------------ 
n2 = length(x2)                              # total scores from series 2
gl2 = n2 - k                                 # denominator degrees freedom
sum_x2, sum_y2 = sum(x2), sum(y2)            # sum of independent and dependent variables from series 2
mean_x2, mean_y2 = sum(x2)/n2, sum(y2)/n2    # average of independent and dependent variables from series 2
SSxy2 = sum((x2.-mean_x2).*(y2.-mean_y2))    # covariance of X2 and Y2
SSxx2 = sum((x2.-mean_x2).^2)                # variance in X2
b2 = SSxy2/SSxx2                             # regression coefficient (series 2)
a2 = mean_y2 - b2*mean_x2                    # intercept (series 2)
yi2 = a2 .+ b2.*x2                           # estimated y value for observation i (series 2)
RSS2 = sum((y2 .- yi2).^2)                   # Residual sum of square after break

233.95555555557337

## [BenchmarkTools.jl](https://juliaci.github.io/BenchmarkTools.jl/stable/)

In [6]:
# Packages requires to load the dataset
#using Pkg
Pkg.add("BenchmarkTools")
using BenchmarkTools

   Resolving package versions...
  No Changes to `C:\Users\andre\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\andre\.julia\environments\v1.7\Manifest.toml`


In [7]:
# Return regression coefficient
function mat_coeff(y, x)
    k = 2
    n = length(x)
    sum_x, sum_y = sum(x), sum(y)
    mean_x, mean_y = sum(x)/n, sum(y)/n
    SSxy = sum((x.-mean_x).*(y.-mean_y))
    SSxx = sum((x.-mean_x).^2)
    b = SSxy/SSxx
    return(b)
end

mat_coeff (generic function with 1 method)

In [8]:
# regression coefficient
reg_coeff(y,x)

LoadError: UndefVarError: reg_coeff not defined

In [9]:
Pkg.add("GLM")
using GLM

   Resolving package versions...
  No Changes to `C:\Users\andre\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\andre\.julia\environments\v1.7\Manifest.toml`


In [10]:
function ols_coeff(Y,X)
    data = DataFrame(X=x, Y=y)
    ols = lm(@formula(Y ~ X), data)
    betas = coef(ols)
    beta1 = betas[2]
    return(beta1)
end

ols_coeff (generic function with 1 method)

In [11]:
ols_coeff(y,x)

1.8897099069232624

In [12]:
# Benchmark - Modelo 1 ("matemático")
@benchmark mat_coeff(y,x)


BenchmarkTools.Trial: 10000 samples with 545 evaluations.
 Range (min … max):  208.624 ns …   3.530 μs  ┊ GC (min … max): 0.00% … 91.67%
 Time  (median):     220.183 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   245.270 ns ± 207.771 ns  ┊ GC (mean ± σ):  6.80% ±  7.49%

  ▅█▇▆▅▄▂▁  ▂▂▁                                                 ▂
  ███████████████▇▆▄▄▃▄▄▅▅▄▃▅▃▅▄▆▄▃▃▃▃▁▄▃▁▁▃▁▁▁▁▁▁▁▄▇▇▆▆▄▄▄▆▆▇▇ █
  209 ns        Histogram: log(frequency) by time        524 ns <

 Memory estimate: 592 bytes, allocs estimate: 3.

In [13]:
# Benchmark - Modelo 2 ("GLM")
@benchmark ols_coeff(y,x)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  16.300 μs …   7.620 ms  ┊ GC (min … max): 0.00% … 98.81%
 Time  (median):     18.900 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   22.668 μs ± 128.952 μs  ┊ GC (mean ± σ):  9.78% ±  1.72%

   ▁█▇▃▁                                                        
  ▂██████▇▆▆▅▄▄▄▄▄▄▄▄▅▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  16.3 μs         Histogram: frequency by time         34.9 μs <

 Memory estimate: 15.05 KiB, allocs estimate: 189.

## Benchmark Solution:

<div align="center">
    <a href="https://pt.wikipedia.org/wiki/Ordens_de_magnitude_para_tempo">Ordens de magnitude para tempo</a>:
</div>

| Fator           | Múltiplo      | Símbolo |
|-----------------|:-------------:|:-------:|
|10<sup>-9</sup>  |1 nanossegundo | ns      |
|10<sup>-6</sup>  |1 microssegundo| µs      |

In [14]:
# Benchmark Time (mean):
Mean_Model1 = 250072*10^-9;
Mean_Model2 = 24871*10^-6;

In [15]:
println("Tempo em segundos para o cálculo do coeficiente:", "\nExpressão Matemática = ", round(Mean_Model1;digits=5),"\nModelo GLM = ", round(Mean_Model2;digits=5))

if Mean_Model1 > Mean_Model2
    println("A função GLM é mais rápida para calcular o coeficiente.")
else
    println("A função 'Matemática' é mais rápida para calcular o coeficiente.")
end

Tempo em segundos para o cálculo do coeficiente:
Expressão Matemática = 0.00025
Modelo GLM = 0.02487
A função 'Matemática' é mais rápida para calcular o coeficiente.


**Referências:**
* [Ciência de Dados e Computação Científica com Julia](https://storopoli.io/Computacao-Cientifica/2_BenchmarkTools/)
* [BenchmarkTools.jl](https://juliaci.github.io/BenchmarkTools.jl/stable/)
* [juliastats.org](https://juliastats.org/GLM.jl/stable/examples/)
* [Julia - GLM Documentation](https://juliastats.org/GLM.jl/v0.11/)
* [Ordens de magnitude para tempo](https://pt.wikipedia.org/wiki/Ordens_de_magnitude_para_tempo)
* [Github Chow_Test Julia](https://github.com/andrelmfsantos/Chow_Test/blob/main/Chow%20Test%20Julia%20-%20Atelier.ipynb)
* [First-time setup: Install Julia & Pluto](https://computationalthinking.mit.edu/Fall20/installation/)